# ADX + Supertrend Swing Scanner (Daily)

This notebook scans a list of tickers and flags *current* swing signals using **Supertrend** for direction
and **ADX** for trend strength. It fetches daily OHLCV with `yfinance`, calculates indicators with pure
NumPy/Pandas (no TA-Lib), and prints a filtered table of stocks that **just flipped** trend today with ADX above a
chosen threshold.

### What you get
- Clean, reusable functions for **Supertrend** and **ADX**
- A single function to **scan** any ticker list
- A final **filtered DataFrame** (Buy/Sell) and CSV export
- IST-aware timestamps (Asia/Kolkata) for clarity

### Strategy rules (from your PDF brief)
- **Buy**: Supertrend flips from bearish→bullish **and** ADX ≥ threshold (default **25**)
- **Sell**: Supertrend flips from bullish→bearish **and** ADX ≥ threshold

> Tip: You can relax/tighten the ADX threshold (e.g., 20 vs 30) to get more/fewer signals.


In [19]:
# === 0) Install deps if needed (uncomment when running locally) ===
# %pip install --upgrade yfinance pandas numpy pytz

import pandas as pd
import numpy as np
import pytz
from typing import List, Dict, Tuple

import yfinance as yf

IST = pytz.timezone('Asia/Kolkata')

pd.set_option('display.width', 180)
pd.set_option('display.max_columns', 50)


## 1) Configuration
Edit the `TICKERS` list (supports NSE, NYSE, etc.), the `PERIOD` / `INTERVAL`, and indicator parameters.


In [20]:
### --- User inputs ---
# Example tickers (mix of NSE & US). Replace with your universe (e.g., NIFTY500).
TICKERS: List[str] = ['ABB.NS', 'ADANIENSOL.NS', 'ADANIGREEN.NS', 'ADANIPOWER.NS', 'AMBUJACEM.NS', 'DMART.NS', 'BAJAJHLDNG.NS', 'BAJAJHFL.NS', 'BANKBARODA.NS', 'BPCL.NS', 'BOSCHLTD.NS', 'BRITANNIA.NS', 'CGPOWER.NS', 'CANBK.NS', 'CHOLAFIN.NS', 'DLF.NS', 'DABUR.NS', 'DIVISLAB.NS', 'GAIL.NS', 'GODREJCP.NS', 'HAVELLS.NS', 'HAL.NS', 'HYUNDAI.NS', 'ICICIGI.NS', 'ICICIPRULI.NS', 'INDHOTEL.NS', 'IOC.NS', 'IRFC.NS', 'NAUKRI.NS', 'INDIGO.NS', 'JSWENERGY.NS', 'JINDALSTEL.NS', 'LTIM.NS', 'LICI.NS', 'LODHA.NS', 'PIDILITIND.NS', 'PFC.NS', 'PNB.NS', 'RECLTD.NS', 'MOTHERSON.NS', 'SHREECEM.NS', 'SIEMENS.NS', 'SWIGGY.NS', 'TVSMOTOR.NS', 'TATAPOWER.NS', 'TORNTPHARM.NS', 'UNITDSPR.NS', 'VBL.NS', 'VEDL.NS', 'ZYDUSLIFE.NS']


# Data settings (daily is typical for swing)
PERIOD   = '1y'   # fetch last 1 year
INTERVAL = '1d'   # daily bars

# Supertrend parameters
ST_ATR_PERIOD = 10
ST_MULTIPLIER = 3.0

# ADX parameters
ADX_PERIOD    = 14
ADX_THRESHOLD = 25.0   # strong trend filter

# Output
OUT_CSV = 'adx_supertrend_signals.csv'


## 2) Indicator implementations (pure Pandas/NumPy)
These match the widely used definitions and are close to Wilder's smoothing via EMA.


In [21]:
def _ema_wilder_like(s: pd.Series, period: int) -> pd.Series:
    """EMA with span=period approximates Wilder smoothing for practical use."""
    return s.ewm(span=period, adjust=False).mean()

def add_supertrend(df: pd.DataFrame, atr_period: int = 10, multiplier: float = 3.0) -> pd.DataFrame:
    """
    Compute Supertrend line and trend state.
    Adds columns: 'ST', 'ST_Trend' (1=uptrend, 0=downtrend).
    """
    df = df.copy()
    h, l, c = df['High'], df['Low'], df['Close']

    # True Range
    tr = pd.concat([
        (h - l),
        (h - c.shift(1)).abs(),
        (l - c.shift(1)).abs()
    ], axis=1).max(axis=1)
    atr = _ema_wilder_like(tr, atr_period)

    hl2 = (h + l) / 2.0
    basic_ub = hl2 + multiplier * atr
    basic_lb = hl2 - multiplier * atr

    fub = pd.Series(index=df.index, dtype='float64')
    flb = pd.Series(index=df.index, dtype='float64')
    st  = pd.Series(index=df.index, dtype='float64')

    for i in range(len(df)):
        if i == 0:
            fub.iat[i] = basic_ub.iat[i]
            flb.iat[i] = basic_lb.iat[i]
            st.iat[i]  = np.nan
            continue

        # Final Upper Band
        prev_fub = fub.iat[i-1]
        fub.iat[i] = basic_ub.iat[i] if (basic_ub.iat[i] < prev_fub) or (c.iat[i-1] > prev_fub) else prev_fub

        # Final Lower Band
        prev_flb = flb.iat[i-1]
        flb.iat[i] = basic_lb.iat[i] if (basic_lb.iat[i] > prev_flb) or (c.iat[i-1] < prev_flb) else prev_flb

        # Supertrend switch logic
        if np.isnan(st.iat[i-1]):
            st.iat[i] = fub.iat[i] if c.iat[i] <= fub.iat[i] else flb.iat[i]
        else:
            if st.iat[i-1] == fub.iat[i-1]:
                st.iat[i] = fub.iat[i] if c.iat[i] <= fub.iat[i] else flb.iat[i]
            else:
                st.iat[i] = flb.iat[i] if c.iat[i] >= flb.iat[i] else fub.iat[i]

    df['ST'] = st
    df['ST_Trend'] = (df['Close'] > df['ST']).astype(int)
    return df

def add_adx(df: pd.DataFrame, period: int = 14) -> pd.DataFrame:
    """
    Compute ADX along with +DI/-DI using Wilder-like EMA.
    Adds: 'ADX', 'DI+','DI-'
    """
    df = df.copy()
    h, l, c = df['High'], df['Low'], df['Close']

    up_move   = h.diff()
    down_move = -l.diff()

    plus_dm  = np.where((up_move > down_move) & (up_move > 0), up_move, 0.0)
    minus_dm = np.where((down_move > up_move) & (down_move > 0), down_move, 0.0)
    plus_dm  = pd.Series(plus_dm, index=df.index)
    minus_dm = pd.Series(minus_dm, index=df.index)

    tr = pd.concat([
        (h - l),
        (h - c.shift(1)).abs(),
        (l - c.shift(1)).abs()
    ], axis=1).max(axis=1)

    atr = _ema_wilder_like(tr, period)
    sm_plus_dm  = _ema_wilder_like(plus_dm, period)
    sm_minus_dm = _ema_wilder_like(minus_dm, period)

    plus_di  = 100 * (sm_plus_dm / atr.replace(0, np.nan))
    minus_di = 100 * (sm_minus_dm / atr.replace(0, np.nan))

    dx = 100 * ( (plus_di - minus_di).abs() / (plus_di + minus_di).replace(0, np.nan) )
    adx = _ema_wilder_like(dx, period)

    df['DI+'] = plus_di
    df['DI-'] = minus_di
    df['ADX'] = adx
    return df


## 3) Data download
Pull daily OHLCV for each ticker via Yahoo! Finance.


In [22]:
def fetch_history(ticker: str, period: str = PERIOD, interval: str = INTERVAL) -> pd.DataFrame:
    df = yf.download(ticker, period=period, interval=interval, auto_adjust=False, progress=False, multi_level_index=False)
    if df is None or df.empty:
        return pd.DataFrame()
    # Make index timezone-aware in IST for clarity in prints/exports
    if df.index.tz is None:
        df.index = df.index.tz_localize('UTC').tz_convert(IST)
    else:
        df.index = df.index.tz_convert(IST)
    return df


## 4) Scanner
Runs indicators and checks the **latest flip** with ADX filter. Returns a DataFrame of signals.


In [23]:
def scan_signals(tickers: List[str],
                 st_atr: int = ST_ATR_PERIOD,
                 st_mult: float = ST_MULTIPLIER,
                 adx_period: int = ADX_PERIOD,
                 adx_threshold: float = ADX_THRESHOLD) -> pd.DataFrame:
    rows = []
    for t in tickers:
        try:
            df = fetch_history(t)
            if df.empty or len(df) < max(st_atr, adx_period) + 2:
                rows.append({'Ticker': t, 'Signal': None, 'Reason': 'insufficient data'})
                continue

            df = add_supertrend(df, atr_period=st_atr, multiplier=st_mult)
            df = add_adx(df, period=adx_period)

            # Need last two rows to detect today's flip
            last, prev = df.iloc[-1], df.iloc[-2]
            flip_up   = (prev['ST_Trend'] == 0) and (last['ST_Trend'] == 1)
            flip_down = (prev['ST_Trend'] == 1) and (last['ST_Trend'] == 0)
            strong    = (last['ADX'] >= adx_threshold)

            signal = None
            if flip_up and strong:
                signal = 'Buy'
            elif flip_down and strong:
                signal = 'Sell'

            rows.append({
                'Ticker': t,
                'Date(IST)': last.name.strftime('%Y-%m-%d %H:%M:%S %Z'),
                'Close': round(float(last['Close']), 4),
                'ADX': round(float(last['ADX']), 2),
                'ST_Trend': int(last['ST_Trend']),
                'Signal': signal,
                'Reason': None if signal else 'no flip / weak ADX'
            })
        except Exception as e:
            rows.append({'Ticker': t, 'Signal': None, 'Reason': f'error: {e}'})

    res = pd.DataFrame(rows)
    res = res.sort_values(['Signal','ADX'], ascending=[True, False]).reset_index(drop=True)
    # Filter only active signals
    filt = res[res['Signal'].notna()].copy()
    return res, filt


## 5) Run the scan
This cell prints both the full table and the **filtered signals**. It also writes a CSV.


In [24]:
all_df, signals_df = scan_signals(TICKERS)
print('--- All results (latest bar) ---')
display(all_df)

print('\n--- Filtered (active signals) ---')
display(signals_df)

signals_df.to_csv(OUT_CSV, index=False)
print(f'\nSaved filtered signals to: {OUT_CSV}')


--- All results (latest bar) ---


,Ticker,Date(IST),Close,ADX,ST_Trend,Signal,Reason
0,VEDL.NS,2025-09-15 05:30:00 IST,454.4000,26.28,1,Buy,None
1,LTIM.NS,2025-09-15 05:30:00 IST,5341.5000,25.76,1,Buy,None
2,TVSMOTOR.NS,2025-09-15 05:30:00 IST,3477.3999,65.35,1,None,no flip / weak ADX
3,BRITANNIA.NS,2025-09-15 05:30:00 IST,6212.0000,64.14,1,None,no flip / weak ADX
4,CGPOWER.NS,2025-09-15 05:30:00 IST,791.3500,58.57,1,None,no flip / weak ADX
5,HYUNDAI.NS,2025-09-15 05:30:00 IST,2549.6001,54.00,1,None,no flip / weak ADX
6,ZYDUSLIFE.NS,2025-09-15 05:30:00 IST,1036.9000,48.18,1,None,no flip / weak ADX
7,MOTHERSON.NS,2025-09-15 05:30:00 IST,107.8100,46.48,1,None,no flip / weak ADX
8,VBL.NS,2025-09-15 05:30:00 IST,471.6500,44.35,0,None,no flip / weak ADX
9,ADANIENSOL.NS,2025-09-15 05:30:00 IST,837.1500,42.97,1,None,no flip / weak ADX



--- Filtered (active signals) ---


,Ticker,Date(IST),Close,ADX,ST_Trend,Signal,Reason
0,VEDL.NS,2025-09-15 05:30:00 IST,454.4,26.28,1,Buy,None
1,LTIM.NS,2025-09-15 05:30:00 IST,5341.5,25.76,1,Buy,None



Saved filtered signals to: adx_supertrend_signals.csv


## 6) Notes & next steps
- Add **position sizing** and **backtests** to evaluate performance across time.
- Consider **+DI > -DI** for extra confirmation on long signals (and vice versa for shorts).
- Tune parameters per asset class; avoid overfitting.

**Happy scanning!**
